In [1]:
import pandas as pd
import numpy as np

from FlagEmbedding import BGEM3FlagModel
import chromadb

import ir_measures
from ir_measures import *

from m3_sentence_transformer.data_sampler import DataSampler

model = BGEM3FlagModel("BAAI/bge-m3", use_fp16=True)
chroma_client = chromadb.PersistentClient(path="../db")
collection = chroma_client.get_collection(name="neuclir-titles-ru-bge-m3")


/Users/I551385/Library/Caches/pypoetry/virtualenvs/ie691-ir-project-u22LL72s-py3.12/lib/python3.12/site-packages/cwl/ruler/cwl_ruler.py:171: SyntaxWarning: invalid escape sequence '\_'
  eval_tool_bibtex = """
/Users/I551385/Library/Caches/pypoetry/virtualenvs/ie691-ir-project-u22LL72s-py3.12/lib/python3.12/site-packages/cwl/ruler/measures/cwl_ift.py:39: SyntaxWarning: invalid escape sequence '\&'
  "booktitle = {The 41st International ACM SIGIR Conference on Research \&\#38; Development in Information Retrieval}," \
/Users/I551385/Library/Caches/pypoetry/virtualenvs/ie691-ir-project-u22LL72s-py3.12/lib/python3.12/site-packages/cwl/ruler/measures/cwl_ift.py:75: SyntaxWarning: invalid escape sequence '\&'
  "booktitle = {The 41st International ACM SIGIR Conference on Research \&\#38; Development in Information Retrieval}," \
/Users/I551385/Library/Caches/pypoetry/virtualenvs/ie691-ir-project-u22LL72s-py3.12/lib/python3.12/site-packages/cwl/ruler/measures/cwl_ift.py:116: SyntaxWarning: i

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

In [2]:
sampler = DataSampler("ru")

In [9]:
print(len(sampler.valid_queries()))

680


In [13]:
sample = sampler.sample_queries(54)

sample.head()

,query_id,doc_id,relevance,iteration,title,description,ht_title,ht_description,mt_title,mt_description,narrative_by_relevance,report,report_url,report_date,translation_lang
0,3,1822ae6f-ad85-4091-8a8c-0e0e5001733c,0,0,British royal news impacts,What political and economic impacts does news ...,Влияние британских королевских новостей,Какое политическое и экономическое влияние нов...,Британские королевские новости влияют,Какие политические и экономические последствия...,{'very_valuable': 'Information regarding econo...,Announcement of engagement\nPrincess Eugenie o...,https://en.wikipedia.org/w/index.php?title=Wed...,2018-05-13,ru
1,3,1b41d078-73fb-41ef-90b9-8fa8be881911,1,0,British royal news impacts,What political and economic impacts does news ...,Влияние британских королевских новостей,Какое политическое и экономическое влияние нов...,Британские королевские новости влияют,Какие политические и экономические последствия...,{'very_valuable': 'Information regarding econo...,Announcement of engagement\nPrincess Eugenie o...,https://en.wikipedia.org/w/index.php?title=Wed...,2018-05-13,ru
2,3,2ef38f45-30d3-4473-b2aa-f498fe707725,0,0,British royal news impacts,What political and economic impacts does news ...,Влияние британских королевских новостей,Какое политическое и экономическое влияние нов...,Британские королевские новости влияют,Какие политические и экономические последствия...,{'very_valuable': 'Information regarding econo...,Announcement of engagement\nPrincess Eugenie o...,https://en.wikipedia.org/w/index.php?title=Wed...,2018-05-13,ru
3,3,4fd2ede3-3ed9-4276-b14c-a54dbba19e95,0,0,British royal news impacts,What political and economic impacts does news ...,Влияние британских королевских новостей,Какое политическое и экономическое влияние нов...,Британские королевские новости влияют,Какие политические и экономические последствия...,{'very_valuable': 'Information regarding econo...,Announcement of engagement\nPrincess Eugenie o...,https://en.wikipedia.org/w/index.php?title=Wed...,2018-05-13,ru
4,3,92d40f5c-7368-4637-856e-c060f8633f8f,0,0,British royal news impacts,What political and economic impacts does news ...,Влияние британских королевских новостей,Какое политическое и экономическое влияние нов...,Британские королевские новости влияют,Какие политические и экономические последствия...,{'very_valuable': 'Information regarding econo...,Announcement of engagement\nPrincess Eugenie o...,https://en.wikipedia.org/w/index.php?title=Wed...,2018-05-13,ru


In [14]:
grouped = sample.groupby("query_id").first().reset_index()

queries = grouped["description"].tolist()

query_embeddings = model.encode(queries, return_dense=True, return_sparse=False, return_colbert_vecs=False)

query_embedding_pairs = list(zip(grouped["query_id"], query_embeddings['dense_vecs']))

len(query_embedding_pairs)

54

In [15]:
results = []
for (q, e) in query_embedding_pairs:
 results.append(collection.query(query_embeddings=e, n_results=1000))

len(results)


54

In [16]:
# Initialize an empty list to hold rows
data = []

# Loop through each query and corresponding results
for i, (q, e) in enumerate(query_embedding_pairs):
    doc_ids = results[i]["ids"][0] # Extract document IDs
    scores = 1 - np.array(results[i]["distances"][0])  # Convert distances to scores
    # Append a row for each doc_id and its corresponding query_id and score
    for doc_id, score in zip(doc_ids, scores):
          data.append({"query_id": q, "doc_id": doc_id, "score": score})

# Convert to a DataFrame
df = pd.DataFrame(data)


In [7]:
df

,query_id,doc_id,score
0,101,4ccdbb8e-a309-49cf-b4a5-98c3b7383ff2,0.643805
1,101,aa109572-7b1b-4dda-85c5-35c5c0ee9e72,0.622699
2,101,1023b31b-eff0-482e-beb5-74218d220a2e,0.618924
3,101,b022741a-f4d2-4c70-8ed0-9a59ff4e7b42,0.592747
4,101,2233a939-8bbc-4aaf-9f89-86b67f95440b,0.583458
...,...,...,...
49995,6,f400c611-84bc-4e05-8d40-e159adabc6ad,0.446992
49996,6,4060f626-c515-4e9c-a35a-a7415a6c47b9,0.446962
49997,6,e99554fc-4c20-4407-bfaa-f7bbbf27579f,0.446915
49998,6,d0905ae0-f326-4281-ac29-efc938dba55a,0.446864


In [20]:

measures = ir_measures.calc_aggregate([nDCG@20, P@5, MRR@10, P(rel=1)@5, Judged@10, MAP, RBP(rel=1), R@100, R@1000], sample[["query_id", "doc_id", "relevance"]], df)

measures

{Judged@10: 0.16481481481481475,
 R@1000: 0.6994789161455828,
 AP: 0.15629194591331602,
 RR@10: 0.2221119929453263,
 R@100: 0.3865119448452781,
 P@5: 0.10740740740740742,
 nDCG@20: 0.20665949674572437,
 RBP(rel=1): np.float64(0.09455150748155751)}